In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [6]:
df_raw = pd.read_csv('C:\\Users\\silri\\Applied_AI\\ML_Classification_Package\\3._Logisitic_Regression\\Facebook_Ads_2.csv',encoding='latin1')

In [7]:
df = df_raw

In [8]:
df.head()

,Names,emails,Country,Time Spent on Site,Salary,Clicked
0,Martina Avila,cubilia.Curae.Phasellus@quisaccumsanconvallis.edu,Bulgaria,25.649648,55330.06006,0
1,Harlan Barnes,eu.dolor@diam.co.uk,Belize,32.456107,79049.07674,1
2,Naomi Rodriquez,vulputate.mauris.sagittis@ametconsectetueradip...,Algeria,20.945978,41098.60826,0
3,Jade Cunningham,malesuada@dignissim.com,Cook Islands,54.039325,37143.35536,1
4,Cedric Leach,felis.ullamcorper.viverra@egetmollislectus.net,Brazil,34.249729,37355.11276,0


**The available columns**

In [9]:
df.columns

Index(['Names', 'emails', 'Country', 'Time Spent on Site', 'Salary',
       'Clicked'],
      dtype='object')

**How to get the idex of a given column**

In [12]:
df.columns.get_loc('Names')

0

- We will create two datasets X and y. X will contain all the features and y will contain only the target variable which is the class label here.

- From the dataset above the name and email will not contribute much to the model.Therefore we can drop those two columns from the dataset and take rest of the columns into account.

In [15]:
X = df.drop(columns=['Names', 'emails','Clicked'])
X

,Country,Time Spent on Site,Salary
0,Bulgaria,25.649648,55330.06006
1,Belize,32.456107,79049.07674
2,Algeria,20.945978,41098.60826
3,Cook Islands,54.039325,37143.35536
4,Brazil,34.249729,37355.11276
...,...,...,...
494,Sao Tome and Principe,19.222746,44969.13495
495,Nepal,22.665662,41686.20425
496,Zimbabwe,35.320239,23989.80864
497,Philippines,26.539170,31708.57054


In [17]:
y = df['Clicked']
y

0      0
1      1
2      0
3      1
4      0
      ..
494    0
495    0
496    0
497    0
498    1
Name: Clicked, Length: 499, dtype: int64

In [21]:
X['Country'].unique().shape

(210,)

In [23]:
## Split the dataset into two halves --> Train and Test data-set

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
X_train, X_test, y_train , y_test = train_test_split(X,y, test_size=0.3, random_state=42)

In [22]:
## Perform OHE on top of the country column
## Perform MinMax on top of Time Spent
## Perform Standard scalling on top of Salary
## Perform Logistic regression on top of X train dataset

In [27]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline,make_pipeline

In [32]:
X.columns

Index(['Country', 'Time Spent on Site', 'Salary'], dtype='object')

In [63]:
X_train

,Country,Time Spent on Site,Salary
461,South Sudan,37.749549,73687.47697
356,Switzerland,39.605587,61820.96760
3,Cook Islands,54.039325,37143.35536
399,Montenegro,39.461526,33843.65733
380,Antarctica,42.644840,69262.62860
...,...,...,...
106,Togo,25.910277,20.00000
270,Bolivia,31.026887,35341.27149
348,Marshall Islands,33.522899,50702.52326
435,Malta,13.194055,24231.08764


In [88]:
oh = OneHotEncoder(drop='first',sparse=False)

In [90]:
oh.fit_transform(X_train).shape

(349, 879)

In [73]:
trf1 = ColumnTransformer([(    
    'MinMax_timeSpent',
    MinMaxScaler(),
    [X.columns.get_loc('Time Spent on Site')])
]
    
    , 
                         remainder = 'passthrough')

In [74]:
trf2 = ColumnTransformer([(    
    'StandardScaling_Salary',
    StandardScaler(),
    [X.columns.get_loc('Salary')])
], 
                         
                         
                         remainder = 'passthrough')

In [75]:
# trf3 = ColumnTransformer([('Encode_country',
#                            OneHotEncoder(drop='first',sparse=False, handle_unknown='ignore' ),
#                           [X.columns.get_loc('Country')]
#                           )]
#                          , remainder='passthrough')

In [76]:
pipe = make_pipeline(trf1, trf2)

In [77]:
X_train

,Country,Time Spent on Site,Salary
461,South Sudan,37.749549,73687.47697
356,Switzerland,39.605587,61820.96760
3,Cook Islands,54.039325,37143.35536
399,Montenegro,39.461526,33843.65733
380,Antarctica,42.644840,69262.62860
...,...,...,...
106,Togo,25.910277,20.00000
270,Bolivia,31.026887,35341.27149
348,Marshall Islands,33.522899,50702.52326
435,Malta,13.194055,24231.08764


In [94]:
pd.get_dummies(df['Country'].unique())

,Afghanistan,Algeria,American Samoa,Andorra,Angola,Anguilla,Antarctica,Argentina,Armenia,Aruba,...,Vanuatu,Venezuela,Viet Nam,"Virgin Islands, British","Virgin Islands, United States",Wallis and Futuna,Western Sahara,Yemen,Zimbabwe,Åland Islands
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
206,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
207,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
208,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [97]:
def oneHotEncoding(df, column):
    df_column = pd.get_dummies(df[column])
    df.drop(column, axis = 1)
    df_final = pd.concat([df,df_column], axis = 1)
    return df_final   

In [99]:
oneHotEncoding(df, 'Country').shape

(499, 216)

In [100]:
df.shape

(499, 6)

In [79]:
pipe.fit_transform(X_train,y_train)

array([[1.030843115268205, 0.576008423807006, 'South Sudan'],
       [0.4218547831367729, 0.611375990290872, 'Switzerland'],
       [-0.844598345617114, 0.8864168592775117, 'Cook Islands'],
       ...,
       [-0.14874291020265962, 0.495467841374196, 'Marshall Islands'],
       [-1.5072549144238623, 0.1080932761896608, 'Malta'],
       [0.06887031749893319, 0.43040948564035725, 'Bhutan']], dtype=object)

In [80]:
pd.DataFrame(pipe.fit_transform(X_train,y_train))

,0,1,2
0,1.030843,0.576008,South Sudan
1,0.421855,0.611376,Switzerland
2,-0.844598,0.886417,Cook Islands
3,-1.013939,0.608631,Montenegro
4,0.80376,0.66929,Antarctica
...,...,...,...
344,-2.749766,0.350406,Togo
345,-0.937081,0.447905,Bolivia
346,-0.148743,0.495468,Marshall Islands
347,-1.507255,0.108093,Malta


In [83]:
len(pd.DataFrame(pipe.fit_transform(X_train,y_train))[2].unique())

184

In [84]:
len(pd.DataFrame(pipe.fit_transform(X_train,y_train))[2])

349

In [85]:
pd.get_dummies(pd.DataFrame(pipe.fit_transform(X_train,y_train))[2])

,Afghanistan,Algeria,American Samoa,Andorra,Angola,Anguilla,Antarctica,Argentina,Armenia,Aruba,...,Uruguay,Vanuatu,Venezuela,Viet Nam,"Virgin Islands, British",Wallis and Futuna,Western Sahara,Yemen,Zimbabwe,Åland Islands
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
345,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
346,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
347,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
